In [6]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 120)

# Dataset Engineering

Input: data/dataset.xlsx
Output: data/dataset_clean.xlsx

Data Cleaning
 - filtering for covid positive patients
 - removing cols with >= 98% missing
Data Engineering:
 - Imputing numerical vars based on mean for col
 - Grouping vars: Either create binary indicator of whether or not a group of tests was completed OR create numerical variable of # of tests from group completed
 - Y var: any changes here?

In [7]:
#original data
df = pd.read_excel("../data/dataset.xlsx")

#filter for only positive patients
df = df[df['SARS-Cov-2 exam result'] == 'positive']
df.head(5)

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,Red blood Cells,Lymphocytes,Mean corpuscular hemoglobin concentration (MCHC),Leukocytes,Basophils,Mean corpuscular hemoglobin (MCH),Eosinophils,Mean corpuscular volume (MCV),Monocytes,Red blood cell distribution width (RDW),Serum Glucose,Respiratory Syncytial Virus,Influenza A,Influenza B,Parainfluenza 1,CoronavirusNL63,Rhinovirus/Enterovirus,Mycoplasma pneumoniae,Coronavirus HKU1,Parainfluenza 3,Chlamydophila pneumoniae,Adenovirus,Parainfluenza 4,Coronavirus229E,CoronavirusOC43,Inf A H1N1 2009,Bordetella pertussis,Metapneumovirus,Parainfluenza 2,Neutrophils,Urea,Proteina C reativa mg/dL,Creatinine,Potassium,Sodium,"Influenza B, rapid test","Influenza A, rapid test",Alanine transaminase,Aspartate transaminase,Gamma-glutamyltransferase,Total Bilirubin,Direct Bilirubin,Indirect Bilirubin,Alkaline phosphatase,Ionized calcium,Strepto A,Magnesium,pCO2 (venous blood gas analysis),Hb saturation (venous blood gas analysis),Base excess (venous blood gas analysis),pO2 (venous blood gas analysis),Fio2 (venous blood gas analysis),Total CO2 (venous blood gas analysis),pH (venous blood gas analysis),HCO3 (venous blood gas analysis),Rods #,Segmented,Promyelocytes,Metamyelocytes,Myelocytes,Myeloblasts,Urine - Esterase,Urine - Aspect,Urine - pH,Urine - Hemoglobin,Urine - Bile pigments,Urine - Ketone Bodies,Urine - Nitrite,Urine - Density,Urine - Urobilinogen,Urine - Protein,Urine - Sugar,Urine - Leukocytes,Urine - Crystals,Urine - Red blood cells,Urine - Hyaline cylinders,Urine - Granular cylinders,Urine - Yeasts,Urine - Color,Partial thromboplastin time (PTT),Relationship (Patient/Normal),International normalized ratio (INR),Lactic Dehydrogenase,"Prothrombin time (PT), Activity",Vitamin B12,Creatine phosphokinase (CPK),Ferritin,Arterial Lactic Acid,Lipase dosage,D-Dimer,Albumin,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
67,78511c183ae18bc,7,positive,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,d7834ed75f2da44,16,positive,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,NaN,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,b16b49f7bd3e692,10,positive,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,NaN,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

## Feature Engineering

Reading in and combining all the binary features we created based on lab test groupings.

In [9]:
#engineered features
virus = pd.read_csv('../data/positive_virus_bac_labs.csv', usecols = range(1,4))
pot_sod = pd.read_csv('../data/pot_sod_grp.csv', usecols = range(1,3))
bili_blood = pd.read_csv('../data/bili-bloodgasses-grp.csv', usecols = range(1,6))
urine = pd.read_csv('../data/urine_data.csv', names = ['Patient ID', 'urine_test'], skiprows = 1)
blood = pd.read_csv('../data/blood_test_id.csv')

#combining all new features
features = virus.merge(pot_sod,on='Patient ID').merge(bili_blood,on='Patient ID').merge(urine,on='Patient ID').merge(blood,on='Patient ID')
features.head()

,Patient ID,virus_lab_taken,num_virus_or_bacteria_dectected,pot_sod_grp,arterial_grp,venous_grp,art_ven_grp,bili_grp,urine_test,blood_test_grp
0,78511c183ae18bc,0,0,0,0,0,0,0,0,0
1,d7834ed75f2da44,1,0,0,0,0,0,0,0,0
2,b16b49f7bd3e692,1,0,0,0,0,0,0,0,0
3,4382f5ea05e60c4,1,0,1,0,1,0,1,0,1
4,d3729cd2658ca64,0,0,0,0,0,0,0,0,0


## Removing columns with missing values

We decided to remove all columns with 98% or more missing values. First we check the proportion of missing values in the entire dataset. Then we remove all the ones with at least 98% missing values.

There are 25 variables we need to remove.

In [10]:
#find columns with 98% or higher values missing
missing = (df.isnull().sum().sort_values(ascending = False)).to_frame(name = 'num')
missing.reset_index(inplace=True)
missing.rename(columns = {'index':'var'}, inplace = True)
missing['prop'] = missing['num']/len(df)
missing[missing.prop>=.98]

,var,num,prop
0,Urine - Nitrite,558,1.000000
1,Albumin,558,1.000000
2,Mycoplasma pneumoniae,558,1.000000
3,Fio2 (venous blood gas analysis),558,1.000000
4,Urine - Sugar,558,1.000000
5,Partial thromboplastin time (PTT),558,1.000000
6,"Prothrombin time (PT), Activity",558,1.000000
7,Vitamin B12,558,1.000000
8,D-Dimer,558,1.000000
9,Phosphor,557,0.998208


In [11]:
#remove above columns from df
remove = list(missing[missing.prop>=.98]['var'])
df.drop(remove, axis = 1, inplace = True)

## Create Y variable

To predict which ward a patient was placed in we need to take the three binary ward variables and create one y variable with 4 classes:
* Admitted to the regular ward
* Admitted to the semi-intensive unit
* Admitted to the intensive care unit
* discharged

In [12]:
conditions = [
    (df['Patient addmited to regular ward (1=yes, 0=no)'] == 1),
    (df['Patient addmited to semi-intensive unit (1=yes, 0=no)'] == 1),
    (df['Patient addmited to intensive care unit (1=yes, 0=no)'] == 1),
    (df['Patient addmited to intensive care unit (1=yes, 0=no)'] == 0) & 
     (df['Patient addmited to semi-intensive unit (1=yes, 0=no)'] == 0) &
     (df['Patient addmited to regular ward (1=yes, 0=no)'] == 0)]

# create a list of the values we want to assign for each condition
values = ['regular', 'semi', 'icu', 'discharged']

# create a new column and use np.select to assign values to it using our lists as arguments
df['y'] = np.select(conditions, values)

#remove unnecessary columns
df.drop(['Patient addmited to regular ward (1=yes, 0=no)',
        'Patient addmited to semi-intensive unit (1=yes, 0=no)',
        'Patient addmited to intensive care unit (1=yes, 0=no)',
        'SARS-Cov-2 exam result'], axis = 1, inplace = True)

df['y'].value_counts()

discharged    506
regular        36
semi            8
icu             8
Name: y, dtype: int64

## Joining in Features

Joining in the clean dataset with the features we engineered.

In [13]:
df = df.merge(features, on = 'Patient ID')

In [14]:
df.head()

,Patient ID,Patient age quantile,Hematocrit,Hemoglobin,Platelets,Mean platelet volume,Red blood Cells,Lymphocytes,Mean corpuscular hemoglobin concentration (MCHC),Leukocytes,Basophils,Mean corpuscular hemoglobin (MCH),Eosinophils,Mean corpuscular volume (MCV),Monocytes,Red blood cell distribution width (RDW),Serum Glucose,Respiratory Syncytial Virus,Influenza A,Influenza B,Parainfluenza 1,CoronavirusNL63,Rhinovirus/Enterovirus,Coronavirus HKU1,Parainfluenza 3,Chlamydophila pneumoniae,Adenovirus,Parainfluenza 4,Coronavirus229E,CoronavirusOC43,Inf A H1N1 2009,Bordetella pertussis,Metapneumovirus,Parainfluenza 2,Neutrophils,Urea,Proteina C reativa mg/dL,Creatinine,Potassium,Sodium,"Influenza B, rapid test","Influenza A, rapid test",Alanine transaminase,Aspartate transaminase,Gamma-glutamyltransferase,Total Bilirubin,Direct Bilirubin,Indirect Bilirubin,Alkaline phosphatase,Strepto A,pCO2 (venous blood gas analysis),Hb saturation (venous blood gas analysis),Base excess (venous blood gas analysis),pO2 (venous blood gas analysis),Total CO2 (venous blood gas analysis),pH (venous blood gas analysis),HCO3 (venous blood gas analysis),Urine - Aspect,Urine - pH,Urine - Hemoglobin,Urine - Bile pigments,Urine - Density,Urine - Urobilinogen,Urine - Leukocytes,Urine - Crystals,Urine - Red blood cells,Urine - Granular cylinders,Urine - Yeasts,Urine - Color,Relationship (Patient/Normal),International normalized ratio (INR),Lactic Dehydrogenase,Creatine phosphokinase (CPK),Arterial Lactic Acid,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),ctO2 (arterial blood gas analysis),y,virus_lab_taken,num_virus_or_bacteria_dectected,pot_sod_grp,arterial_grp,venous_grp,art_ven_grp,bili_grp,urine_test,blood_test_grp
0,78511c183ae18bc,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discharged,0,0,0,0,0,0,0,0,0
1,d7834ed75f2da44,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,regular,1,0,0,0,0,0,0,0,0
2,b16b49f7bd3e692,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discharged,1,0,0,0,0,0,0,0,0
3,4382f5ea05e60c4,2,0.991838,0.792188,-0.341548,1.469188,1.653476,-0.048383,-0.452899,-0.420197,1.303529,-1.442245,-0.498393,-1.396114,1.933339,0.967144,NaN,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,-0.414215,-0.513909,NaN,-0.32234,-0.058626,0.143752,NaN,NaN,-0.504127,-0.317035,-0.330608,1.355535,1.163312,1.198484,3.060642,NaN,-0.090035,0.337027,-0.611396,-0.084646,-0.479346,-0.436537,-0.512865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000994,0.86241,-0.620717,0.125483,NaN,